In [2]:
# Notebook: Answers to Question 2
# Author: Ben Bernal
# Date: 08JUL2024

### Preliminaries

#### Libraries

In [3]:
import numpy as np
import pandas as pd
import networkx as nx
from os import path, listdir
from pprint import pprint
import re
import plotly.express as px
import plotly.graph_objects as go
from dataclasses import make_dataclass
import statsmodels.tsa.api as smt

#### Utilities

In [4]:
from utilities import json as util_json

#### Configuration

In [5]:
env_config = util_json.to_dict(
    file_path="../../config/env.json"
)

# Dataset

In [24]:
raw_frame = pd.read_csv(
    path.normpath(
        path.join(
            env_config['root'],
            "modules/m4/release_time_series_report_data_nops/CO2.csv"
        )
    ),
    header=56,
    names = [
        'year',
        'month',
        'date_excel',
        'date_decimal',
        'CO2_[ppm]',
        'CO2_[ppm]_seasonality_adjusted',
        "CO2_[ppm]_fit",
        "CO2_[ppm]_seasonality_adjusted_fit",
        "CO2_[ppm]_filled",
        "CO2_[ppm]_seasonality_adjusted_filled"
    ]
)

print(raw_frame.head(3).to_markdown(tablefmt="grid"))

+----+--------+---------+--------------+----------------+-------------+----------------------------------+-----------------+--------------------------------------+--------------------+-----------------------------------------+
|    |   year |   month |   date_excel |   date_decimal |   CO2_[ppm] |   CO2_[ppm]_seasonality_adjusted |   CO2_[ppm]_fit |   CO2_[ppm]_seasonality_adjusted_fit |   CO2_[ppm]_filled |   CO2_[ppm]_seasonality_adjusted_filled |
+====+========+=========+==============+================+=============+==================================+=================+======================================+====================+=========================================+
|  0 |   1958 |       1 |        21200 |        1958.04 |      -99.99 |                           -99.99 |          -99.99 |                               -99.99 |             -99.99 |                                  -99.99 |
+----+--------+---------+--------------+----------------+-------------+---------------------